In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Window
import pandas as pd
import pandas_redshift as pr

In [2]:
# pip install pandas_redshift
spark = SparkSession.builder.getOrCreate()

In [3]:
pr.connect_to_redshift(
    host= 'hostname',
    user = 'username',
    port = 'port',
    password = '####',
    dbname='dbname'
)

In [4]:
sql_statement = 'select * from DriveDim_Type2_Hamzah'
sql_statement1 = 'select * from driverfeed_hamzah'
sql_statement2 = 'select * from shifttimedim_hamzah'
sql_statement3 = 'select * from coveragedim_hamzah'
sql_statement4 = 'select * from companydim_hamzah'


In [5]:
df = pr.redshift_to_pandas(sql_statement)
df1 = pr.redshift_to_pandas(sql_statement1)
df2 = pr.redshift_to_pandas(sql_statement2)
df3 = pr.redshift_to_pandas(sql_statement3)
df4 = pr.redshift_to_pandas(sql_statement4)

In [6]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [7]:
#These are the five dataframes
driverdim = spark.createDataFrame(df)
driverfeed = spark.createDataFrame(df1)
shiftTime = spark.createDataFrame(df2)
coverages = spark.createDataFrame(df3)
companys = spark.createDataFrame(df4)

In [8]:
from datetime import datetime, timedelta
maxx = driverdim.agg({"driverkey": "max"}).head(1)[0][0]

In [9]:
change = driverdim.join(driverfeed, (driverdim.drivername==driverfeed.drivername), how='inner')
change = change.select([driverdim.driverrecordkey, driverdim.driverkey, driverdim.drivername, driverdim.shifttimekey, driverdim.coveragekey, driverdim.companykey, driverdim.hiredate, driverdim.firedate, driverdim.effectivedate, driverdim.loaddate, driverdim.expireddate, driverdim.current_record])
nochange = driverdim.subtract(change)

In [10]:
nochange.show()

+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+--------+-----------+--------------+
|driverrecordkey|driverkey|drivername|shifttimekey|coveragekey|companykey|hiredate|  firedate|effectivedate|loaddate|expireddate|current_record|
+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+--------+-----------+--------------+
|             70|       68|   L19Name|           3|          3|         3|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 12/31/9999|             1|
|             22|       20|   D19Name|           3|          3|         1|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 12/31/9999|             1|
|             19|       17|   D16Name|           2|          8|         1|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 12/31/9999|             1|
|             83|       81|   S08Name|           1|          8|         4|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 12/31/9999| 

In [11]:
def dateMinusOne(s):
    date = datetime.strptime(s, "%m/%d/%Y")
    modified_date = date - timedelta(days=1)
    return datetime.strftime(modified_date, "%m/%d/%Y")
udfModifyTime = F.udf(dateMinusOne, T.StringType())

In [12]:
def firedTypeTwo(driverdim, driverfeed):
    drivedimFired = driverdim.join(driverfeed, (driverdim.drivername == driverfeed.drivername) & (driverfeed.action == 'FIRED'), how='inner')
    drivedimFired = drivedimFired.select([driverdim.driverrecordkey, driverdim.driverkey, driverdim.drivername, driverdim.shifttimekey, driverdim.coveragekey, driverdim.companykey, driverdim.hiredate, driverdim.firedate, driverdim.effectivedate, driverdim.loaddate, driverdim.expireddate, driverdim.current_record, driverfeed.effdt, driverfeed.filedt])
    drivedimFired = drivedimFired.withColumn('expireddate', udfModifyTime('effdt'))
    drivedimFired = drivedimFired.withColumn('current_record', F.when(F.col('current_record') == 1, 0)).persist()
    newRecords = drivedimFired.select(['driverrecordkey','driverkey', 'drivername', 'shifttimekey', 'coveragekey', 'companykey', 'hiredate', 'firedate', 'effectivedate', 'loaddate', 'expireddate', 'current_record', 'effdt', 'filedt'])
    newRecords = newRecords.withColumn('firedate', F.lit(F.col('effdt')))
    newRecords = newRecords.withColumn('effectivedate', F.lit(F.col('effdt')))
    newRecords = newRecords.withColumn('loaddate', F.lit(F.col('filedt')))
    newRecords = newRecords.withColumn('current_record', F.lit(1))
    newRecords = newRecords.withColumn('expireddate', F.lit('12/31/9999'))
    newRecords = newRecords.withColumn('driverrecordkey', F.lit('999'))
    return drivedimFired.unionAll(newRecords)

driverFiredAndAdded = firedTypeTwo(driverdim, driverfeed)
driverFiredAndAdded = driverFiredAndAdded.select(['driverrecordkey', 'driverkey','drivername', 'shifttimekey', 'coveragekey','companykey','hiredate', 'firedate', 'effectivedate', 'loaddate', 'expireddate', 'current_record'])


In [13]:
driverFiredAndAdded.show()

+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+----------+-----------+--------------+
|driverrecordkey|driverkey|drivername|shifttimekey|coveragekey|companykey|hiredate|  firedate|effectivedate|  loaddate|expireddate|current_record|
+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+----------+-----------+--------------+
|             15|       13|   D12Name|           2|          4|         1|7/1/2014|12/31/9999|     7/1/2014|  7/2/2014| 10/18/2014|             0|
|             25|       23|   D22Name|           3|          6|         1|7/1/2014|12/31/9999|     7/1/2014|  7/2/2014| 10/04/2014|             0|
|             87|       85|   S12Name|           2|          4|         4|7/1/2014|12/31/9999|     7/1/2014|  7/2/2014| 11/08/2014|             0|
|             79|       77|   S04Name|           1|          4|         4|7/1/2014|12/31/9999|     7/1/2014|  7/2/2014

In [14]:
def firedTypeTwo(driverdim, driverfeed, shiftTime, coverages, companys):
    

    drivedimChanged = driverdim.join(driverfeed, (driverdim.drivername == driverfeed.drivername) & (driverfeed.action == 'CHANGE'), how='inner').join(shiftTime, (driverfeed.beginshift==shiftTime.starttime) & (driverfeed.endshift==shiftTime.endtime), how='inner').join(coverages, (driverfeed.coverage==coverages.coverage), how='inner').join(companys, (driverfeed.source==companys.company), how='inner')
    drivedimChanged = drivedimChanged.select([driverdim.driverrecordkey, driverdim.driverkey, driverdim.drivername,shiftTime.shifttimekey, coverages.coveragekey,companys.compnaykey, driverdim.hiredate, driverdim.firedate, driverdim.effectivedate, driverdim.loaddate, driverdim.expireddate, driverdim.current_record, driverfeed.effdt, driverfeed.filedt])
    drivedimChanged = drivedimChanged.withColumn('expireddate', udfModifyTime('effdt'))   
    drivedimChanged = drivedimChanged.withColumn('current_record', F.when(F.col('current_record') == 1, 0)).persist()
    changedRecords = drivedimChanged.select(['driverrecordkey','driverkey', 'drivername', 'shifttimekey', 'coveragekey', 'compnaykey', 'hiredate', 'firedate', 'effectivedate', 'loaddate', 'expireddate', 'current_record', 'effdt', 'filedt'])
    changedRecords = changedRecords.withColumn('effectivedate', F.lit(F.col('effdt')))
    changedRecords = changedRecords.withColumn('loaddate', F.lit(F.col('filedt')))
    changedRecords = changedRecords.withColumn('current_record', F.lit(1))
    changedRecords = changedRecords.withColumn('firedate', F.lit('12/31/9999'))
    changedRecords = changedRecords.withColumn('expireddate', F.lit('12/31/9999'))
    return drivedimChanged.unionAll(changedRecords)

driverChangedAndAdded = firedTypeTwo(driverdim, driverfeed, shiftTime, coverages, companys)
driverChangedAndAdded = driverChangedAndAdded.select(['driverrecordkey','driverkey', 'drivername', 'shifttimekey', 'coveragekey', 'compnaykey', 'hiredate', 'firedate', 'effectivedate', 'loaddate', 'expireddate', 'current_record']).persist()


In [15]:
driverChangedAndAdded.show()

+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+--------+-----------+--------------+
|driverrecordkey|driverkey|drivername|shifttimekey|coveragekey|compnaykey|hiredate|  firedate|effectivedate|loaddate|expireddate|current_record|
+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+--------+-----------+--------------+
|             29|       27|   F02Name|           1|          8|         2|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 11/02/2014|             0|
|             28|       26|   F01Name|           1|          2|         2|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 11/02/2014|             0|
|             38|       36|   F11Name|           2|          6|         2|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 11/02/2014|             0|
|             46|       44|   F19Name|           3|          1|         2|7/1/2014|12/31/9999|     7/1/2014|7/2/2014| 11/02/2014| 

In [16]:
def hiredTypeTwo(driverfeed, shiftTime, coverages, companys):
    drivedimHired = driverfeed.join(shiftTime, (driverfeed.action == 'HIRED') & (driverfeed.beginshift==shiftTime.starttime) & (driverfeed.endshift==shiftTime.endtime), how='inner').join(coverages, (driverfeed.coverage==coverages.coverage), how='inner').join(companys, (driverfeed.source==companys.company), how='inner')
    drivedimHired = drivedimHired.withColumn('key', F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))
    drivedimHired = drivedimHired.withColumn('driverkey', maxx+drivedimHired.key)
    drivedimHired = drivedimHired.withColumn('effectivedate', F.lit(F.col('effdt')))
    drivedimHired = drivedimHired.withColumn('hiredate', F.lit(F.col('effdt')))
    drivedimHired = drivedimHired.withColumn('loaddate', F.lit(F.col('filedt')))
    drivedimHired = drivedimHired.withColumn('firedate', F.lit('12/31/9999'))
    drivedimHired = drivedimHired.withColumn('expireddate', F.lit('12/31/9999'))
    drivedimHired = drivedimHired.withColumn('current_record', F.lit(1))
    drivedimHired = drivedimHired.withColumn('driverrecordkey', F.lit(999))
    drivedimHired = drivedimHired.select('driverrecordkey', 'driverkey', driverfeed.drivername,shiftTime.shifttimekey, coverages.coveragekey,companys.compnaykey,'hiredate', 'firedate', 'effectivedate', 'loaddate', 'expireddate', 'current_record')
    return drivedimHired
driverHired = hiredTypeTwo(driverfeed, shiftTime, coverages, companys).persist()

In [17]:
final_table = nochange.union(driverFiredAndAdded).union(driverChangedAndAdded).union(driverHired).persist()

In [18]:
final_table = final_table.orderBy('driverkey', ascending=True)

In [19]:
final_table = final_table.withColumn('driverrecordkey', F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))

In [20]:
final_table.show()

+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+----------+-----------+--------------+
|driverrecordkey|driverkey|drivername|shifttimekey|coveragekey|companykey|hiredate|  firedate|effectivedate|  loaddate|expireddate|current_record|
+---------------+---------+----------+------------+-----------+----------+--------+----------+-------------+----------+-----------+--------------+
|              1|       -1|      NULL|          -1|         -1|        -1|1/1/1900|12/31/9999|     1/1/1900|  1/1/1900| 12/31/9999|             1|
|              2|        1|   D00Name|           1|          1|         1|1/1/2014| 6/30/2014|    6/30/2014|  7/1/2014| 12/31/9999|             1|
|              3|        1|   D00Name|           1|          1|         1|1/1/2014|12/31/9999|     1/1/2014|  1/2/2014|  6/29/2014|             0|
|              4|        2|   D01Name|           1|          1|         1|7/1/2014|12/31/9999|     7/1/2014|  7/2/2014

In [ ]:
final_table.coalesce(1).write.csv("driverFinal_dim_Type2.csv")

In [ ]:
# pr.close_up_shop()